<a href="https://colab.research.google.com/github/Buyan-Kirill/practice-VTM-sem-8/blob/main/Tensors_task_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Развёртка тензора
def unfold(tensor, ax=0):
    if ax < 0 or ax >= len(tensor.shape):
        raise ValueError("Недопустимый номер направления")
    tensor_1 = np.moveaxis(tensor, ax, 0)
    return tensor_1.reshape((tensor.shape[ax], -1))

# Умножение по направлению
def mode_product(matrix, tensor, ax):
    if matrix.shape[1] != tensor.shape[ax]:
        raise ValueError(f"Размер матрицы {matrix.shape} не совместим размером {ax} тензора {tensor.shape} в данном направление")
    result = np.tensordot(matrix, tensor, axes=(1, ax))
    if ax != 0:
        result = np.moveaxis(result, 0, ax)
    return result

In [2]:
def st_ho_svd(T: np.array, error):
    U_matrices = []
    total_error = 0 # Оценка ошибки (должна совападть с настоящей ошибкой)
    if hasattr(error, "__len__"):
        if len(error) != len(T.shape):
            print(len(T.shape))
            raise ValueError('Incorrect error size')
        errors = error
    else:
        errors = [error / len(T.shape)] * len(T.shape)
    X = T
    for d in range(len(T.shape)):
        U_i, D_i, V_i = np.linalg.svd(unfold(X, d), full_matrices=False)
        k = 0
        error_i = np.linalg.norm(D_i) ** 2
        while error_i > errors[d] and k < len(D_i):
            error_i -= D_i[k]**2
            k += 1
        total_error += error_i
        U_matrices.append(U_i[:, :k])
        X = mode_product(U_matrices[d].T, X, d)
    return X, U_matrices, total_error

In [5]:
def test_hosvd(tensor: np.array, core: np.array, U: list[np.array], estimation=None):
    t_appr = core
    for d in range(len(tensor.shape)):
        t_appr = mode_product(U[d], t_appr, d)
    print("Tensor shapes = ", tensor.shape)
    print("Core shapes = ", core.shape)
    print("True error = ", np.linalg.norm(tensor - t_appr) ** 2)
    if estimation:
        print("Error estimation = ", estimation)
    print()


In [6]:
np.random.seed(42)
# test 1
T = np.array([[[1], [2]], [[3], [4]], [[5], [6]]])
core, U, err = st_ho_svd(T, 1)
test_hosvd(T, core, U, err)


# test 2
T = np.array([[[1, 1, 2], [1, 2, 5]], [[0, 2, 1], [3, 5, 0]], [[4, 2, 1], [3, 1, 0]]])
core, U, err = st_ho_svd(T, 1)
test_hosvd(T, core, U, err)


# test 3
T = np.random.rand(10, 6, 8, 5)
core, U, err = st_ho_svd(T, [20, 0, 20, 0])
test_hosvd(T, core, U, err)


# test 4
T = np.random.rand(10, 60, 28, 15)
core, U, err = st_ho_svd(T, 0)
test_hosvd(T, core, U, err)


# test 5
T = np.random.rand(10, 60, 28, 15)
core, U, err = st_ho_svd(T, [144, 361, 256, 100])
test_hosvd(T, core, U, err)


# test 5
T = np.random.rand(80, 4, 5, 3, 8)
core, U, err = st_ho_svd(T, [2000, 10, 10, 10, 10])
test_hosvd(T, core, U, err)


Tensor shapes =  (3, 2, 1)
Core shapes =  (1, 1, 1)
True error =  0.2645050872658184
Error estimation =  0.26450508726583166

Tensor shapes =  (3, 2, 3)
Core shapes =  (3, 2, 3)
True error =  1.3661682367071843e-29
Error estimation =  2.4868995751603507e-14

Tensor shapes =  (10, 6, 8, 5)
Core shapes =  (9, 6, 7, 5)
True error =  32.998366979129926
Error estimation =  32.998366979130346

Tensor shapes =  (10, 60, 28, 15)
Core shapes =  (10, 60, 28, 15)
True error =  2.3019043951930664e-25
Error estimation =  -1.0231815394945443e-12

Tensor shapes =  (10, 60, 28, 15)
Core shapes =  (10, 59, 28, 15)
True error =  276.7948917091043
Error estimation =  276.7948917091402

Tensor shapes =  (80, 4, 5, 3, 8)
Core shapes =  (19, 4, 5, 3, 8)
True error =  1998.0117257541922
Error estimation =  1998.0117257541967

